In [87]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from nltk import tokenize
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

d_set = pd.read_csv(r"C:\Users\pc\Desktop\spam.csv", encoding="ISO-8859-1")
label_encoder = LabelEncoder()
d_set['v1'] = label_encoder.fit_transform(d_set['v1'])
#removing null values
d_set = d_set.dropna(axis=1, how='any') 

# lowercasing
d_set['v2'] = d_set['v2'].str.lower()

stop_words = set(stopwords.words('english'))
# cleaning data 
def preprocess_text(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    text = re.sub(r'\d+', '', text)
    text = " ".join([word for word in text.split() if word.lower() not in stop_words])
    tokens = word_tokenize(text)
    
    return tokens  

# Apply to your dataframe
d_set['v3'] = d_set['v2'].apply(preprocess_text)
d_set

,v1,v2,v3
0,0,"go until jurong point, crazy.. available only ...","[go, jurong, point, crazy, available, bugis, n..."
1,0,ok lar... joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,1,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, wkly, comp, win, fa, cup, final,..."
3,0,u dun say so early hor... u c already then say...,"[u, dun, say, early, hor, u, c, already, say]"
4,0,"nah i don't think he goes to usf, he lives aro...","[nah, dont, think, goes, usf, lives, around, t..."
...,...,...,...
5567,1,this is the 2nd time we have tried 2 contact u...,"[nd, time, tried, contact, u, u, å£, pound, pr..."
5568,0,will ì_ b going to esplanade fr home?,"[ì, b, going, esplanade, fr, home]"
5569,0,"pity, * was in mood for that. so...any other s...","[pity, mood, soany, suggestions]"
5570,0,the guy did some bitching but i acted like i'd...,"[guy, bitching, acted, like, id, interested, b..."


In [89]:
# single string
d_set['v3'] = d_set['v3'].apply(lambda x: ' '.join(x))
tfid = TfidfVectorizer(max_features=3000)
X = tfid.fit_transform(d_set['v3']).toarray()
X_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
import joblib
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X_array, d_set['v1'], test_size=0.2, random_state = 2)

# bayes
model = BernoulliNB()
model.fit(X_train, y_train)

joblib.dump(model, 'spam_detection.joblib')
joblib.dump(tfid, 'spam_count.joblib')
#y_pred = model.predict(X_test)
#score = accuracy_score(y_test, y_pred)
#display('X-Train :', X_train.shape)
#display('X-Test :',X_test.shape)
#display('Y-Train :',y_train.shape)
#display('X-Test :',y_test.shape)

'X-Train :'

(4457, 8460)

'X-Test :'

(1115, 8460)

'Y-Train :'

(4457,)

'X-Test :'

(1115,)